In [ ]:
keys = [
 '2404/D19-M3-S0-L8/5'
# '2404/D21-M3-S0-L8/6',
# '2404/D25-M3-S0-L8/111',
# '2404/D25-M3-S0-L8/97',
# '2404/D25-M3-S0-L8/120',
# '2404/D25-M3-S0-L8/125',
# '2404/D25-M3-S0-L8/139',
# '2404/D25-M3-S0-L8/143',
# '2405/D18-M3-S0-L8/13',
# '2405/D18-M3-S0-L8/14',
# '2405/D18-M3-S0-L8/15',
# '2405/D18-M3-S0-L8/10',
# '2405/D18-M3-S0-L8/11',
# '2405/D18-M3-S0-L8/12',
]

gt_dir_root = '/group/jug/ashesh/kth_data'
pred_dir = '/group/jug/ashesh/data/paper_stats/Test_P64_G32_M50_Sk16_F'
output_dir = '/group/jug/ashesh/ForDamianSegmentation/'


In [ ]:
import os
from disentangle.core.tiff_reader import load_tiff
import numpy as np
import tifffile

def save_tiff(fpath, data):
    tifffile.imwrite(fpath, data.transpose(0,3,1,2),
    imagej=True, 
    metadata={ 'axes': 'ZCYX'}, 
    )

def get_dtype_subdir(key):
    return key.strip('/').split('/')[1].split('-')[0]

def pred_subfpath(key, k):
    # input: '2404/D21-M3-S0-L8/6'
    # output: pred_disentangle_2404_D21-M3-S0-L8_6_1.tif
    return f"kth_{k}/pred_disentangle_{key.replace('/', '_')}_1.tif"

def gt_subfpath(key, k):
    path = pred_subfpath(key, k).replace(f'kth_{k}/', f'kth{k}/gt_for_') 
    return os.path.join(get_dtype_subdir(key), path)



def get_agg_data(key):
    gt_data_list= []
    pred_data_list = []

    for kth in [0,1,2,3,4]:
        gt_fpath = os.path.join(gt_dir_root, gt_subfpath(key, kth))
        pred_fpath = os.path.join(pred_dir, pred_subfpath(key, kth))
        gt_data = load_tiff(gt_fpath)
        pred_data = load_tiff(pred_fpath)
        if gt_data.shape[0] > 10:
            assert len(gt_data.shape)  <= 3
            gt_data = gt_data[None]
        
        if pred_data.shape[0] > 10:
            assert len(pred_data.shape)  <= 3
            pred_data = pred_data[None]

        gt_data_list.append(gt_data)
        pred_data_list.append(pred_data)

    return np.concatenate(gt_data_list, axis=0), np.concatenate(pred_data_list, axis=0)


In [ ]:
import matplotlib.pyplot as plt
key = keys[0]
gt, pred = get_agg_data(key)
_,ax = plt.subplots(figsize=(15,10),ncols=3,nrows=2)
ax[0,1].imshow(gt[0,...,0])
ax[0,2].imshow(gt[0,...,1])

ax[1,1].imshow(pred[0,...,0])
ax[1,2].imshow(pred[0,...,1])
try:
    ax[0,0].imshow(gt[0,...,2])
except:
    print("No input channel")


### Saving it to output

In [ ]:
from tqdm import tqdm
for key in tqdm(keys):
    gt, pred = get_agg_data(key)
    output_subdir = os.path.join(output_dir,key.replace('/','_'))
    os.makedirs(output_subdir, exist_ok=True)
    output_gt_fpath = os.path.join(output_subdir, 'GT.tif')
    output_pred_fpath = os.path.join(output_subdir, os.path.basename(pred_subfpath(key, 0)))
    save_tiff(output_gt_fpath, gt)
    save_tiff(output_pred_fpath, pred.astype(np.float32))
    print(f"Saved to {output_subdir}")

In [ ]:
! ls /group/jug/ashesh/ForDamianSegmentation/